In [1]:
# Import standard library modules
import sys

# Set the relative path to the project root directory
relative_path_to_root = "../../../"

# Add the project root to the system path for importing in-house modules
sys.path.append(relative_path_to_root)

# Import in-house modules from the 'utilities' package
from utilities import calculate_bollinger_bands, calculate_rsi, calculate_daily_volatility
from utilities import print_title, print_label, print_footer
from utilities import generate_trading_signals
from utilities import fetch_and_download_sp500_data, sp500_data_for_today
from utilities import print_dataframe_report, save_data, load_data

In [2]:
# Import libraries for data analysis and visualization
import pandas as pd

# Import libraries for signal processing and peak detection
from scipy.signal import find_peaks
import numpy as np

# Import libraries for plotting and visualization
import matplotlib.pyplot as plt
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

# Import libraries for data storage and retrieval
from pathlib import Path

In [3]:
file_name = "sp500_adj_close_raw"
file_path = f"../../../data/raw_data/{file_name}"
loaded_data = load_data(file_path)


╔═══════════════════════════════════════════════════════════════╗
║File `sp500_adj_close_raw.csv.bz2` loaded from `sp500_adj_close_raw.zip`║
╚═══════════════════════════════════════════════════════════════╝


In [4]:
display(loaded_data.head())
display(loaded_data.tail())

,Date,Ticker,Adjusted Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
0,2008-01-02,A,23.256384,-0.009917,0.015704,48.827570,23.314173,23.299887,23.564932,24.727251,22.540230,21.392027,24.351934,short
1,2008-01-02,AAPL,5.876339,0.000462,0.018937,59.067200,5.518483,4.939064,4.197630,6.135834,5.403558,4.637376,6.026839,buy
2,2008-01-02,ABT,18.130207,-0.006092,0.010484,34.677666,18.138458,17.628250,17.709028,19.233107,18.221802,16.775558,19.134012,short
3,2008-01-02,ACGL,7.608889,0.020444,0.016022,45.154190,7.785511,7.878933,7.874161,8.114465,7.378535,7.463333,8.307778,buy
4,2008-01-02,ACN,26.437075,-0.017194,0.024039,54.812140,26.577982,27.784420,28.471031,28.227200,24.273777,24.765509,29.215677,sell


,Date,Ticker,Adjusted Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
1979674,2024-10-23,XYL,131.74,0.002817,0.009287,45.930984,133.56926,134.558320,130.65660,137.970140,131.13087,126.71,137.53000,buy
1979675,2024-10-23,YUM,134.01,0.004874,0.010931,33.050730,134.76096,133.576890,133.94536,140.494570,130.85544,129.71,139.92000,buy
1979676,2024-10-23,ZBH,104.68,0.004028,0.010726,51.860280,107.59999,108.161766,115.56048,108.536896,101.40522,101.77,115.91237,buy
1979677,2024-10-23,ZBRA,368.08,-0.010538,0.009870,55.442924,354.97920,335.774200,309.98975,378.896670,362.71933,320.77,377.68000,short
1979678,2024-10-23,ZTS,188.99,-0.002744,0.010509,45.437954,189.21500,183.135120,179.39548,196.476970,186.50803,180.90,196.48000,sell


In [5]:
ticker_df = loaded_data.copy()

filter_ticker = ticker_df["Ticker"] == "A"
ticker_df = ticker_df[filter_ticker].reset_index(drop=True)


check_columns = ["Date", "Ticker", "Adjusted Close", "Action"]
ticker_df[check_columns].head(30)

,Date,Ticker,Adjusted Close,Action
0,2008-01-02,A,23.256384,short
1,2008-01-03,A,23.025740,short
2,2008-01-04,A,22.276157,buy
3,2008-01-07,A,22.641344,sell
4,2008-01-08,A,22.609304,buy
5,2008-01-09,A,22.666970,hold
6,2008-01-10,A,23.685640,sell
7,2008-01-11,A,23.064188,buy
8,2008-01-14,A,23.474216,sell
9,2008-01-15,A,23.057774,short
